![CMCC](https://scontent.fcuz1-1.fna.fbcdn.net/v/t1.6435-9/169999414_134997141968151_5389622841264143457_n.jpg?_nc_cat=100&ccb=1-5&_nc_sid=09cbfe&_nc_eui2=AeGYHrPMueJlavl7eDql2QOI2St2L7MLFHTZK3YvswsUdPzk3Ovp__UvRhUQlcFyH2o&_nc_ohc=KZJCq4jFuhMAX9h5HUI&_nc_ht=scontent.fcuz1-1.fna&oh=00_AT_sGFdClC9xMtR4S8p-ilZYOR4ycFv1uCTqoKj-frAi8g&oe=61F5FE4C)
# **Algoritmos pre-procesamiento: PySpark**
```
NOMBRE: JORGE ANDRE SALCEDO HURTADO
CURSO: MINERIA DE DATOS
DOCENTE: CARLOS FERNANDO MONTOYA CUBAS
```

#### Implementar cinco algoritmos a su elección de las diapositivas de pre-procesamiento en un notebook Spark.


# 1. Inicializamos nuestra variable de pyspark

In [1]:
from pyspark import SparkContext
sc =SparkContext()

22/01/09 11:47:16 WARN Utils: Your hostname, MR resolves to a loopback address: 127.0.1.1; using 192.168.1.20 instead (on interface enp9s0)
22/01/09 11:47:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/09 11:47:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 2. Implementación

## 2.1. Similitud de cosenos 
## $$ D(x,y) = \frac{x \cdot y}{\|x\| \cdot \|y\|} $$
La similitud coseno es una medida de la similitud existente entre dos vectores en un espacio que posee un producto interior con el que se evalúa el valor del coseno del ángulo comprendido entre ellos. 

In [2]:
import numpy as np
# vamos a generar una RDD que representan dos vectores numéricos en forma de tuplas.
np.random.seed(28)
vectores_tuplaRDD = sc.parallelize(np.random.random(size=(16,2)),2)

In [3]:
# mostramos los vectores
vectores_tuplaRDD.collect()

[array([0.72901374, 0.5612396 ]),
 array([0.12496709, 0.39759237]),
 array([0.78130821, 0.51099298]),
 array([0.18269336, 0.85351288]),
 array([0.95537189, 0.98421347]),
 array([0.19270097, 0.9707951 ]),
 array([0.23480835, 0.02635385]),
 array([0.94606034, 0.92172485]),
 array([0.29397577, 0.1662737 ]),
 array([0.39542284, 0.51066973]),
 array([0.30803723, 0.42956883]),
 array([0.83006941, 0.56239357]),
 array([0.83088831, 0.99692929]),
 array([0.33257881, 0.09100813]),
 array([0.77383156, 0.14938373]),
 array([0.72535506, 0.95514643])]

In [4]:
# hallamos el valor de la normal de nuestros vectores, 
# que es la suma de los valores elevados al cuadrado del vector.
# Estos valores utilizaremos para el denominador de nuestra función
valueNormVectorX = vectores_tuplaRDD.map(lambda tupla: np.power(tupla[0],2)).sum()
valueNormVectorY = vectores_tuplaRDD.map(lambda tupla: np.power(tupla[1],2)).sum()
# multiplicamos los valores
multNormal = valueNormVectorX * valueNormVectorY

In [5]:
# realizamos el PRODUCTO PUNTO de los vectores, utilizando nuestro vectores_tuplaRD,
# que facilita el acceso para poder multiplicar los valores de cada vector encontrados en
# la misma posicion, para luego sumarlos
pPuntoXY = vectores_tuplaRDD.map(lambda tupla: tupla[0]*tupla[1]).sum()

In [6]:
# Finalmente hallamos la similitud, dividiendo nuestro producto punto de los vectores, 
# y la multiplicacion de los valores normales de ambos vectores
similitudCoseno = pPuntoXY/multNormal
print(similitudCoseno)

0.1318127717148206


## 2.2. Estandarización
## $$\hat{X}_{i,j}=\frac{X_{i,j}-\bar{X}_j}{\sigma_j}$$

In [7]:
# vamos a generar una RDD de 4 particiones, que representará el vector que estandarizaremos
np.random.seed(14)
vectorRDD = sc.parallelize(np.random.random(size=(20)),4)

In [8]:
# mostramos el vector
vectorRDD.collect()

[0.5139433437726902,
 0.7731650520792968,
 0.8704276857248127,
 0.00804694852981791,
 0.3097359255052041,
 0.9576037393658909,
 0.5131167122678871,
 0.3182844248720986,
 0.5391999374256481,
 0.2212549424277599,
 0.80648135792427,
 0.34225462561468434,
 0.5388888490671446,
 0.00587378555105833,
 0.6731524781805254,
 0.21002426217873815,
 0.9325575928787119,
 0.3742447494814921,
 0.7524189220859464,
 0.763139001845522]

In [9]:
# Obtenemos la media del vector
media = vectorRDD.mean()
print(media)

0.52119071683896


In [10]:
# Obtenemos la varianza del vector
var = vectorRDD.map(lambda x: np.power((x - media),2)).mean()
print(var)

0.08087649216270273


In [11]:
# Obtenemos la desviación estándar del arreglo
desviacion = var ** 0.5
print(desviacion)

0.28438792548682995


In [12]:
# Finalmente estandarizamos los valores, utilizando la funcion de estandarizacion como funcion anonima lambda
vectorStandarizadoRDD = vectorRDD.map(lambda x: (x-media)/desviacion)
# Mostramos el arreglo con un escalonamiento de sus valores
print(vectorStandarizadoRDD.collect())

[-0.025484109614932988, 0.8860233246857934, 1.2280302276828763, -1.8043795897125945, -0.7435434924734468, 1.5345694504429348, -0.028390813559511646, -0.7134842016218371, 0.06332624901658171, -1.0546712695264913, 1.0031742402457302, -0.629197216857796, 0.06223236165138881, -1.812021134180558, 0.5343467416256487, -1.094161976559135, 1.4464990921662824, -0.5167095864070818, 0.8130732162807475, 0.850768486715399]


## 2.3. Escalonamiento
Los algoritmos basados en métodos de gradiente tienden a beneficiarse cuando los atributos están entre [0, 1].
## $$\hat{X}_{i,j}=\frac{X_{i,j}-maxX_j}{maxX_j-minX_j} $$


In [13]:
def Escalonamiento(RDDnumeros):
    """Devuelve los valores de un vector escalonados.

    Args:
        RDDnumeros (list of double): Una lista de números.

    Returns:
        list if (double): Valores de la lista escalonados.
    """
    maxNum = max(RDDnumeros.collect())
    minNum = min(RDDnumeros.collect())
    return(RDDnumeros
            .map(lambda x: ((x-minNum)/(maxNum-minNum)))
           )


In [14]:
import numpy as np
# vamos a generar una RDD de 4 particiones, que representará el vector que escalaremos
np.random.seed(32)
vectorRDD = sc.parallelize(np.random.random(size=(20)),4)

In [15]:
vectorRDD.collect()

[0.8588892672930397,
 0.37271115497527996,
 0.5551287779985967,
 0.9556565489917677,
 0.7366695980112238,
 0.8162051391124808,
 0.10108656030862473,
 0.928488070407937,
 0.6091091689913093,
 0.5965534361232749,
 0.09178413479620207,
 0.34518624330535763,
 0.6627525231855876,
 0.4417134884612358,
 0.5514877863939606,
 0.7037124914270836,
 0.5894012313015157,
 0.049932759415546646,
 0.5617918399347968,
 0.7663584723742113]

In [16]:
VectorEscalonado = Escalonamiento(vectorRDD).collect()

In [17]:
print(VectorEscalonado)

[0.8931602737916331, 0.3563761924711705, 0.5577815492948752, 1.0, 0.7582188372428579, 0.8460331819874857, 0.056478367336483926, 0.9700035718433072, 0.6173807246880372, 0.6035180736099319, 0.04620765829750067, 0.32598623033624535, 0.6766077813378106, 0.43256093475142066, 0.5537615692010104, 0.7218312464967205, 0.5956214003591325, 0.0, 0.5651381650897608, 0.7909980075646164]


## 2.4 Normalizacion
La normalización va a transformar las características de forma que todas compartan un mismo valor medio y una misma desviación media
## $$\hat{X}_{i,j}=\frac{X_{i,j}}{||X_i||p}$$

In [18]:
import numpy as np
def Normalizacion(RDDnumeros):
    """Devuelve los valores de un vector normalizados.

    Args:
        RDDnumeros (list of double): Una lista de números.

    Returns:
        list if (double): Valores de la lista normalizados.
    """
    nx = np.sqrt(RDDnumeros.map(lambda x: np.power(x,2)).sum())
    return(RDDnumeros
            .map(lambda x: x/nx)
           )

In [19]:
# vamos a generar una RDD de 4 particiones, que representará el vector que normalizaremos
np.random.seed(64)
vectorRDD = sc.parallelize(np.random.random(size=(20)),4)
vectorRDD.collect()

[0.3790985254337701,
 0.567098167179614,
 0.5955925126571219,
 0.4498588988554526,
 0.45701982539471864,
 0.3116509978753955,
 0.3133373611975704,
 0.29485946100158045,
 0.5307724612356264,
 0.3962220215994189,
 0.7187503721849594,
 0.5480919969498905,
 0.582075082707141,
 0.16765093779104812,
 0.029277197597395443,
 0.20874191791401042,
 0.022575436577678576,
 0.6346744421866868,
 0.2244417391997393,
 0.398309870301092]

In [20]:
VectorNormalizado = Normalizacion(vectorRDD).collect()

In [21]:
print(VectorNormalizado)

[0.19465413443970092, 0.2911855242601669, 0.30581639666729027, 0.23098716745603207, 0.23466405845868682, 0.16002213453426484, 0.1608880244568205, 0.1514002543189702, 0.27253351594558084, 0.20344646440429798, 0.369053747744842, 0.2814265055173552, 0.29887565844164793, 0.0860830259004051, 0.015032840211183372, 0.10718183961898169, 0.011591715007591359, 0.32588363157965583, 0.11524316119688756, 0.20451850334562652]


## 2.5 Distancia de Minkowski
Distancia entre dos vectores numéricos:
$$D_p(x,y)=\sqrt[p]{\sum_{i=1}^{N} |x_i-y_i|^p}$$

In [22]:
def minkowski(RDDnumeros, p):
    """Devuelve la distancia de Minkowski de dos vectores de numeros.

    Args:
        RDDnumeros (list of double): Una lista de dos vectores de numeros.
        p (int): El poder a utilizar en el cálculo de la distancia de Minkowski
        cuando p = 1, nos referimos a la distancia de Manhattan
        cuando p = 2, nos referimos a la distancia Euclidiana

    Returns:
        numero (double): Distancia entre dos vectores numéricos.
    """
    summation = RDDnumeros.map(lambda x: np.power(np.abs(x[0]-x[1]),2)).sum()
    return(np.power(summation,1/p))


In [23]:
# vamos a generar una RDD de 4 particiones, que representará los vectores a los cuales hallaremos su distancia
np.random.seed(10)
vectorRDD = sc.parallelize(np.random.random(size=(20,2)),4)
vectorRDD.collect()

[array([0.77132064, 0.02075195]),
 array([0.63364823, 0.74880388]),
 array([0.49850701, 0.22479665]),
 array([0.19806286, 0.76053071]),
 array([0.16911084, 0.08833981]),
 array([0.68535982, 0.95339335]),
 array([0.00394827, 0.51219226]),
 array([0.81262096, 0.61252607]),
 array([0.72175532, 0.29187607]),
 array([0.91777412, 0.71457578]),
 array([0.54254437, 0.14217005]),
 array([0.37334076, 0.67413362]),
 array([0.44183317, 0.43401399]),
 array([0.61776698, 0.51313824]),
 array([0.65039718, 0.60103895]),
 array([0.8052232 , 0.52164715]),
 array([0.90864888, 0.31923609]),
 array([0.09045935, 0.30070006]),
 array([0.11398436, 0.82868133]),
 array([0.04689632, 0.62628715])]

In [24]:
VectorDistancia = minkowski(vectorRDD,3)

In [25]:
print(VectorDistancia)

1.4664295719169498
